In [1]:
%matplotlib inline
''' Python Tools '''
import numpy as np
np.set_printoptions(suppress=True) # Changes print functionality
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

'''PyTorch Functionality'''
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

'''Landscape Class'''
from Protein_Landscape.landscape_class import Protein_Landscape

'''Dataset Management'''
Datasets = 'Data/'


In [2]:
test = Protein_Landscape(csv_path=Datasets+"Small_NK.csv",gen_graph=True)


Building Protein Graph for entire dataset


  0%|          | 0/1000 [00:00<?, ?it/s]

{(0, 0, 0): 0, (0, 0, 1): 1, (0, 0, 2): 2, (0, 0, 3): 3, (0, 0, 4): 4, (0, 0, 5): 5, (0, 0, 6): 6, (0, 0, 7): 7, (0, 0, 8): 8, (0, 0, 9): 9, (0, 1, 0): 10, (0, 1, 1): 11, (0, 1, 2): 12, (0, 1, 3): 13, (0, 1, 4): 14, (0, 1, 5): 15, (0, 1, 6): 16, (0, 1, 7): 17, (0, 1, 8): 18, (0, 1, 9): 19, (0, 2, 0): 20, (0, 2, 1): 21, (0, 2, 2): 22, (0, 2, 3): 23, (0, 2, 4): 24, (0, 2, 5): 25, (0, 2, 6): 26, (0, 2, 7): 27, (0, 2, 8): 28, (0, 2, 9): 29, (0, 3, 0): 30, (0, 3, 1): 31, (0, 3, 2): 32, (0, 3, 3): 33, (0, 3, 4): 34, (0, 3, 5): 35, (0, 3, 6): 36, (0, 3, 7): 37, (0, 3, 8): 38, (0, 3, 9): 39, (0, 4, 0): 40, (0, 4, 1): 41, (0, 4, 2): 42, (0, 4, 3): 43, (0, 4, 4): 44, (0, 4, 5): 45, (0, 4, 6): 46, (0, 4, 7): 47, (0, 4, 8): 48, (0, 4, 9): 49, (0, 5, 0): 50, (0, 5, 1): 51, (0, 5, 2): 52, (0, 5, 3): 53, (0, 5, 4): 54, (0, 5, 5): 55, (0, 5, 6): 56, (0, 5, 7): 57, (0, 5, 8): 58, (0, 5, 9): 59, (0, 6, 0): 60, (0, 6, 1): 61, (0, 6, 2): 62, (0, 6, 3): 63, (0, 6, 4): 64, (0, 6, 5): 65, (0, 6, 6): 66, (0, 

100%|██████████| 1000/1000 [00:00<00:00, 84593.28it/s]

Calculating the number of extrema

        Protein Landscape class
            Number of Sequences : 1000
            Max Distance        : 3
            Number of Distances : 3
            Seed Sequence       : AAA
                Modified positions are shown in green
            Number of minima : 1
            Number of maxima : 1
            Normalized Extrema Ruggedness : 0.002
            R/S Ruggedness : 17.31943397828702
        


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  20,  30,  40,
        50,  60,  70,  80,  90, 100, 200, 300, 400, 500, 600, 700, 800,
       900])

In [4]:
test.build_graph(idxs=test.random_walk_data())

Building Protein Graph For subset of length 486236


100%|██████████| 1001/1001 [00:00<00:00, 12337.86it/s]

[0, 900, 900, 400, 470, 470, 270, 273, 275, 270, 770, 770, 770, 770, 774, 774, 774, 774, 779, 779, 779, 779, 777, 777, 779, 778, 748, 748, 748, 746, 766, 266, 260, 265, 285, 285, 265, 215, 205, 205, 105, 605, 602, 602, 602, 602, 602, 602, 682, 682, 692, 592, 512, 516, 516, 516, 516, 516, 519, 510, 560, 560, 560, 360, 360, 362, 352, 352, 332, 332, 332, 338, 338, 368, 368, 358, 358, 358, 358, 358, 358, 358, 358, 358, 658, 654, 654, 654, 654, 654, 654, 954, 953, 953, 957, 957, 957, 927, 927, 927, 922, 992, 912, 952, 952, 951, 951, 951, 955, 935, 935, 935, 905, 405, 205, 235, 235, 235, 235, 235, 235, 235, 835, 835, 837, 837, 637, 537, 537, 531, 531, 531, 530, 532, 532, 532, 532, 432, 432, 412, 412, 412, 413, 413, 413, 415, 515, 515, 513, 519, 519, 519, 519, 518, 518, 518, 518, 518, 518, 518, 578, 574, 544, 574, 554, 554, 554, 754, 154, 154, 254, 254, 264, 269, 269, 263, 263, 163, 143, 140, 640, 640, 642, 662, 162, 162, 162, 142, 122, 522, 525, 725, 795, 775, 775, 775, 775, 775, 785, 385, 3

{0: {'tokenized': (0, 0, 0),
  'string': 'AAA',
  'fitness': 0.660972597708149,
  'neighbours': array([  7,   9,  10,  30,  40,  50,  70,  80, 100, 400, 600, 900])},
 900: {'tokenized': (9, 0, 0),
  'string': 'LAA',
  'fitness': 0.6135006022532151,
  'neighbours': array([  0, 100, 400, 600, 905, 907])},
 400: {'tokenized': (4, 0, 0),
  'string': 'FAA',
  'fitness': 0.6878297111016926,
  'neighbours': array([  0, 100, 405, 408, 410, 430, 450, 470, 480, 490, 600, 900])},
 470: {'tokenized': (4, 7, 0),
  'string': 'FIA',
  'fitness': 0.5325536367501192,
  'neighbours': array([ 70, 170, 270, 370, 400, 410, 430, 450, 475, 477, 480, 490, 570,
         670, 770])},
 270: {'tokenized': (2, 7, 0),
  'string': 'DIA',
  'fitness': 0.7326371798927745,
  'neighbours': array([ 70, 170, 260, 273, 275, 278, 370, 470, 570, 670, 770])},
 273: {'tokenized': (2, 7, 3),
  'string': 'DIE',
  'fitness': 0.5058854046983936,
  'neighbours': array([263, 270, 275, 278, 973])},
 275: {'tokenized': (2, 7, 5),
  's

(1,
 array([  0,   2,   3,   4,   5,   6,   7,   8,   9,  11,  21,  31,  41,
         51,  61,  71,  81,  91, 101, 201, 301, 401, 501, 601, 701, 801,
        901]))

In [7]:
type(test.deep_sequence_data()[0])

numpy.int64